# Consolidación de Datos Demográficos por AGEB - Distrito Electoral 11, Puebla

## **Propósito del Proyecto**

Esta libreta tiene como objetivo **consolidar y procesar información demográfica de AGEBs (Áreas Geoestadísticas Básicas)** del Distrito Electoral Federal 11 de Puebla, preparando los datos para análisis electoral y estudios sociodemográficos.

## **Contexto y Problemática**

### **Desafío Principal:**
- **No existe información pública oficial** que relacione directamente:
  -  Datasets con **información electoral** (INE)
  -  Datasets con **información demográfica** (INEGI)

### **Solución Implementada:**
- **Clasificación manual** de correspondencia geográfica entre:
  - **Secciones electorales** del Distrito 11
  - **AGEBs** con información demográfica
- Proceso realizado **"a ojo"** utilizando mapas oficiales del **INE** e **INEGI**
- Registro manual de qué secciones electorales se encuentran dentro de los límites geográficos de cada AGEB

### **Desafío Técnico Adicional:**
- **INEGI proporciona datasets separados** para cada AGEB individual
- **Necesidad de reorganizar** toda la información en un **dataset único consolidado**

---

## **Procedimiento Implementado**

### **1. Extracción y Consolidación de Datos**
- **Lectura automática** de 123 archivos Excel individuales (uno por AGEB)
- **Extracción de secciones electorales** contenidas en cada AGEB
- **Consolidación** de todas las variables demográficas en un dataset único
- **Normalización** de nombres de variables para consistencia

### **2. Limpieza y Optimización del Dataset**
- **Conversión a formato numérico** de todas las variables
- **Eliminación de duplicados**: Remoción de columnas absolutas cuando existe la versión relativa
- **Conservación selectiva** de variables absolutas importantes para análisis electoral
- **Reducción dimensional**: De 683 variables iniciales a 355 variables optimizadas

### **3. Tratamiento de Valores Faltantes**
- **Imputación inteligente** basada en el contexto de cada variable:
  - Variables demográficas específicas → `0` (ceros estructurales)
  - Variables de carencias de servicios → `0` (ausencia probable)
  - Variables generales → **mediana** (método robusto)

---

## **Resultado Final**

**Dataset consolidado** con:
- **122 AGEBs** del Distrito Electoral 11
- **355 variables** demográficas optimizadas
- **Correspondencia manual** con secciones electorales
- **Datos completamente numéricos** listos para análisis


In [1]:
#Importando librerias
import os
import pandas as pd
from openpyxl import load_workbook
import unicodedata
import re
import csv
import numpy as np

In [2]:
# Listar archivos Excel en DatosPorAGEB
excel_files = [f for f in os.listdir('DatosPorAGEB') if f.endswith('.xlsx')]
print(f"Total de archivos Excel encontrados: {len(excel_files)}")
    
# Variables para almacenar resultados
todas_las_secciones = set()  # Usar set para evitar repeticiones
archivos_con_secciones = []
archivos_procesados = 0

Total de archivos Excel encontrados: 123


In [3]:
#En esta celda se extraen las secciones de cada AGEB
for file in excel_files:
        try:
            file_path = os.path.join('DatosPorAGEB', file)
            xl = pd.ExcelFile(file_path)
            archivos_procesados += 1
            
            # Buscar hojas que contengan información de secciones
            for sheet in xl.sheet_names:
                if 'seccion' in sheet.lower() or 'secciones' in sheet.lower():
                    df_sheet = pd.read_excel(file_path, sheet_name=sheet)
                    
                    # Buscar columnas que puedan contener números de sección
                    for col in df_sheet.columns:
                        if 'seccion' in str(col).lower() or 'secc' in str(col).lower():
                            # Extraer valores únicos de esa columna
                            secciones_archivo = set(df_sheet[col].dropna().astype(str))
                            todas_las_secciones.update(secciones_archivo)
                            archivos_con_secciones.append(file)
                            #print(f"{file} - {sheet} - {col}: {len(secciones_archivo)} secciones")
                            break
                    break
            
        except Exception as e:
            print(f"Error en {file}: {e}")
    

In [4]:
# Mostrar y guardar la lista completa de secciones
if todas_las_secciones:
    #print(f"\n LISTA COMPLETA DE SECCIONES SIN REPETICIONES:")
    print("=" * 60)
        
    # Ordenar las secciones numéricamente si es posible
    try:
        secciones_ordenadas = sorted(list(todas_las_secciones), key=lambda x: int(x) if x.isdigit() else x)
    except:
        secciones_ordenadas = sorted(list(todas_las_secciones))
        
    print(f"Total de secciones únicas: {len(secciones_ordenadas)}")
    
    """
    print("\nLista de secciones:")

    for i in range(0, len(secciones_ordenadas), 5):
        fila = secciones_ordenadas[i:i+5]
        numeros = [f"{i+j+1:3d}." for j in range(len(fila))]
        secciones = [f"{seccion:>8}" for seccion in fila]
        print("  ".join([f"{num} {sec}" for num, sec in zip(numeros, secciones)]))
    """ 
    # También guardar como CSV para uso posterior
    df_secciones = pd.DataFrame({
        'numero': range(1, len(secciones_ordenadas) + 1),
        'seccion': secciones_ordenadas
     })
    df_secciones.to_csv('secciones_unicas.csv', index=False, encoding='utf-8')
    print(f"Lista guardada en 'secciones_unicas.csv'")
        
else:
    print("\n No se encontraron secciones en los archivos revisados.")

Total de secciones únicas: 167
Lista guardada en 'secciones_unicas.csv'


### Reorganizacion de los datos en un mismo dataset

In [5]:

def normalizar_nombre_variable(nombre):
    """
    Normaliza el nombre de la variable: quita acentos, reemplaza ñ por n, elimina caracteres especiales,
    y convierte a mayúsculas SIN aplicar abreviaturas.
    """
    if not nombre or not isinstance(nombre, str):
        return "VAR_DESCONOCIDA"
    
    # Quitar acentos
    nombre = unicodedata.normalize('NFD', nombre)
    nombre = nombre.encode('ascii', 'ignore').decode('utf-8')
    
    # Reemplazar ñ por n
    nombre = nombre.replace('ñ', 'n').replace('Ñ', 'N')
    
    # Convertir a mayúsculas
    nombre = nombre.upper()
    
    # Limpiar caracteres especiales
    nombre = re.sub(r'[^A-Z0-9_]', '_', nombre)
    nombre = re.sub(r'_+', '_', nombre)
    nombre = nombre.strip('_')
    
    return nombre

def extraer_secciones(workbook):
    """
    Extrae los códigos de secciones de la hoja 'seccion'
    """
    secciones = []
    if 'seccion' in workbook.sheetnames:
        ws = workbook['seccion']
        for row in ws.iter_rows(min_row=2, values_only=True):
            if row[0] and str(row[0]).strip():
                secciones.append(str(row[0]).strip())
    return secciones

def es_valor_valido(valor):
    """
    Verifica si un valor es válido (no None, no vacío, no string vacío)
    """
    if valor is None:
        return False
    if isinstance(valor, str) and valor.strip() == '':
        return False
    return True

def extraer_indicadores(workbook):
    """
    Extrae indicadores de todas las hojas excepto 'seccion'
    Aplica la lógica específica para columnas absolutas/relativas:
    - Si ambas columnas existen y tienen datos válidos: crear dos columnas con sufijos _ABSOLUTO y _RELATIVO
    - Si solo una columna existe o tiene datos válidos: crear una columna sin sufijo
    """
    indicadores = {}
    
    for sheet_name in workbook.sheetnames:
        if sheet_name.lower() != 'seccion':
            ws = workbook[sheet_name]
            
            # Buscar la fila que contiene "Indicador", "Absoluto", "Relativo"
            fila_encabezados = None
            for row_num, row in enumerate(ws.iter_rows(values_only=True), 1):
                if row and any('indicador' in str(cell).lower() if cell else False for cell in row):
                    fila_encabezados = row_num
                    break
            
            if fila_encabezados:
                # Extraer datos desde la siguiente fila hasta la penúltima
                for row_num in range(fila_encabezados + 1, ws.max_row):
                    row = list(ws.iter_rows(min_row=row_num, max_row=row_num, values_only=True))[0]
                    
                    if row and row[0] and str(row[0]).strip():
                        nombre_indicador = str(row[0]).strip()
                        valor_absoluto = row[1] if len(row) > 1 else None
                        valor_relativo = row[2] if len(row) > 2 else None
                        
                        # Normalizar nombre del indicador
                        nombre_normalizado = normalizar_nombre_variable(nombre_indicador)
                        
                        # Verificar validez de valores
                        abs_valido = es_valor_valido(valor_absoluto)
                        rel_valido = es_valor_valido(valor_relativo)
                        
                        # Aplicar lógica según especificaciones del usuario
                        if abs_valido and rel_valido:
                            # Ambos valores válidos: crear dos columnas con sufijos
                            indicadores[f"{nombre_normalizado}_ABSOLUTO"] = valor_absoluto
                            indicadores[f"{nombre_normalizado}_RELATIVO"] = valor_relativo
                        elif abs_valido and not rel_valido:
                            # Solo absoluto válido: crear columna sin sufijo
                            indicadores[nombre_normalizado] = valor_absoluto
                        elif not abs_valido and rel_valido:
                            # Solo relativo válido: crear columna sin sufijo
                            indicadores[nombre_normalizado] = valor_relativo
                        # Si ninguno es válido, no agregar nada
    
    return indicadores

In [6]:

def procesar_archivos_excel():
    """
    Procesa todos los archivos Excel en la carpeta DatosPorAGEB
    """
    carpeta = "DatosPorAGEB"
    datos_consolidados = []
    
    # Procesar cada archivo Excel
    for archivo in os.listdir(carpeta):
        if archivo.endswith('.xlsx'):
            ruta_archivo = os.path.join(carpeta, archivo)
            #print(f"Procesando: {archivo}")
            
            try:
                workbook = load_workbook(ruta_archivo, data_only=True)
                
                # Extraer código AGEB del nombre del archivo
                ageb_code = archivo.replace('.xlsx', '')
                
                # Extraer indicadores
                indicadores = extraer_indicadores(workbook)
                
                # Extraer secciones
                secciones = extraer_secciones(workbook)
                
                # Crear fila de datos
                fila_datos = {'AGEB': ageb_code}
                fila_datos.update(indicadores)
                
                # Agregar información de secciones
                if secciones:
                    fila_datos['SECCIONES_CONTENIDAS'] = ';'.join(secciones)
                    fila_datos['NUM_SECCIONES'] = len(secciones)
                else:
                    fila_datos['SECCIONES_CONTENIDAS'] = ''
                    fila_datos['NUM_SECCIONES'] = 0
                
                datos_consolidados.append(fila_datos)
                
            except Exception as e:
                print(f"Error procesando {archivo}: {e}")
    
    return datos_consolidados

def guardar_resultados(datos_consolidados):
    """
    Guarda los resultados en archivo CSV
    """
    # Guardar datos consolidados
    if datos_consolidados:
        with open('AGEB_Consolidado_Completo.csv', 'w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=datos_consolidados[0].keys())
            writer.writeheader()
            writer.writerows(datos_consolidados)
        
        print(f"Archivo consolidado guardado: AGEB_Consolidado_Completo.csv")
        print(f"Total de AGEBs procesados: {len(datos_consolidados)}")
        #print(f"Total de variables: {len(datos_consolidados[0]) - 1}")  # -1 por AGEB

In [7]:
print("Iniciando procesamiento completo de archivos AGEB...")
datos = procesar_archivos_excel()
guardar_resultados(datos)
print("Procesamiento completado.")

Iniciando procesamiento completo de archivos AGEB...
Archivo consolidado guardado: AGEB_Consolidado_Completo.csv
Total de AGEBs procesados: 123
Procesamiento completado.


### Limpieza del dataset

In [8]:
new_df = pd.read_csv('AGEB_Consolidado_Completo.csv')

In [9]:
# Obtener todas las columnas excepto 'AGEB'
columnas_numericas = [col for col in new_df.columns if col != 'AGEB']

# Convertir cada columna a float, los strings se convertirán automáticamente a NaN
for col in columnas_numericas:
    new_df[col] = pd.to_numeric(new_df[col], errors='coerce')


In [10]:
#Eliminando SECCIONES_CONTENIDAS
new_df = new_df.drop(columns=['SECCIONES_CONTENIDAS'])

In [11]:
#Droppeando columnas con mas del 50% de datos nulos
new_df = new_df.dropna(axis=1, thresh=0.5*len(new_df))
#Droppeando rows con mas del 50% de datos nulos
new_df = new_df.dropna(axis=0, thresh=0.5*len(new_df))

In [12]:
#viendo top 10 rows con mas nulos
#new_df.isnull().sum(axis=1).sort_values(ascending=False).head(10)

In [13]:
#viendo top 10 columnas con mas nulos
#new_df.isnull().sum().sort_values(ascending=False).head(10)

In [14]:
def eliminar_columnas_absolutas_duplicadas(df):
    """
    Elimina columnas con valores absolutos cuando existe también su versión relativa,
    conservando solo las columnas absolutas especificadas como excepciones.
    
    Args:
        df: DataFrame de pandas
    
    Returns:
        DataFrame con columnas absolutas duplicadas eliminadas
    """
    # Columnas absolutas que se deben conservar siempre
    conservar_absolutos = [
        'ELECTORAL_TOTAL_VOTOS',
        'ELECTORAL_LISTA_NOMINAL',
        'GRADO_PROMEDIO_DE_ESCOLARIDAD',
        'EDAD_MEDIANA_DE_LA_POBLACION_TOTAL'
    ]
    
    # Hacer una copia para no modificar el original
    df_limpio = new_df.copy()
    
    # Encontrar todas las columnas que terminan en "_ABSOLUTO"
    columnas_absolutas = [col for col in df_limpio.columns if col.endswith('_ABSOLUTO')]
    
    columnas_a_eliminar = []
    
    for col_absoluta in columnas_absolutas:
        # Generar el nombre de la columna relativa correspondiente
        base_name = col_absoluta.replace('_ABSOLUTO', '')
        col_relativa = base_name + '_RELATIVO'
        
        # Si existe la versión relativa y no está en la lista de conservar
        if col_relativa in df_limpio.columns and col_absoluta not in conservar_absolutos:
            columnas_a_eliminar.append(col_absoluta)
            #print(f"Eliminando: {col_absoluta} (existe versión relativa)")
        #elif col_absoluta in conservar_absolutos:
            #print(f"Conservando: {col_absoluta} (en lista de excepciones)")
    
    # Eliminar las columnas identificadas
    df_limpio = df_limpio.drop(columns=columnas_a_eliminar)
    

    print(f"- Columnas absolutas encontradas: {len(columnas_absolutas)}")
    print(f"- Columnas absolutas eliminadas: {len(columnas_a_eliminar)}")
    print(f"- Columnas absolutas conservadas: {len(columnas_absolutas) - len(columnas_a_eliminar)}")
    print(f"- Total de columnas después de limpieza: {len(df_limpio.columns)}")
    
    return df_limpio


In [15]:
df_limpio = eliminar_columnas_absolutas_duplicadas(new_df)

- Columnas absolutas encontradas: 328
- Columnas absolutas eliminadas: 328
- Columnas absolutas conservadas: 0
- Total de columnas después de limpieza: 355


In [16]:
def limpiar_e_imputar_dataset(df):
    """
    Función concisa para limpiar dataset y aplicar imputación inteligente
    """
    df_clean = df.copy()
    # Imputación inteligente y concisa
    # Variables que deben ser 0 cuando falta (grupos muy específicos)
    cero_patterns = ['85_anos', 'lengua_indigena', 'afromexicana', 'afrodescendiente', 
                     'pemex', 'bienestar', 'otra_institucion', 'colectivas']
    
    # Variables de vivienda sin servicios (probable que sea 0)
    sin_servicios = ['sin_ningun_bien', 'no_disponen', 'piso_de_tierra', 
                     'solo_un_cuarto', 'sin_tecnologias', 'sin_radio', 'sin_linea']
    
    for col in df_clean.select_dtypes(include=[np.number]).columns:
        col_lower = col.lower()
        
        # Estrategia 1: Ceros estructurales
        if any(pattern in col_lower for pattern in cero_patterns + sin_servicios):
            df_clean[col] = df_clean[col].fillna(0)
        
        # Estrategia 2: Mediana para el resto
        else:
            mediana = df_clean[col].median()
            df_clean[col] = df_clean[col].fillna(mediana)
    
    # 3. Reporte
    nulos_inicial = df.isnull().sum().sum()
    nulos_final = df_clean.isnull().sum().sum()
    
    print(f" Nulos eliminados: {nulos_inicial:,} → {nulos_final:,}")
    print(f" Dataset final: {df_clean.shape[0]} filas × {df_clean.shape[1]} columnas")
    print(f" {df_clean.select_dtypes(include=[np.number]).shape[1]} variables numéricas")
    
    return df_clean



In [17]:
df_limpio = limpiar_e_imputar_dataset(df_limpio)

 Nulos eliminados: 930 → 0
 Dataset final: 122 filas × 355 columnas
 354 variables numéricas


In [18]:
#guardando el df limpio
df_limpio.to_csv('AGEB_Consolidado_Completo.csv', index=False, encoding='utf-8')